## Walkthrough for annotating a mutation list with database data using annovar and clinscore

#### Oftentimes you receive a mutation list of any sort and you need to annotate it with database info to have an idea about the position and relevance of the data. More than often, this task can be subdivided into several or all of the following steps:
+ wrangle the data into the format (ID) Chr Start End Ref Alt .....
+ if coords are on hg19, convert it to hg38 to use this annotation set (same procedure can be applied if converting back to hg19 is required+ annotate with annovar on command line using any of a set of databases
+ apply clinscore calculation to get relevance of mutations in a clear (scalar) fashion

#### init paths and code base

In [34]:
# some sensible settings for better output
import os
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('max_colwidth', 200)


# get the code
import sys
sys.path.append('../code')
from script_utils import show_output

# paths from environ file
home = os.environ['HOME']
# you need static files for the annovar annotation
static_path = os.path.join(os.environ['STATIC'], "annotation/clinical")
# here, set the working directory where your files are saved

workdir = os.path.join("../")

### load mutation file and inspect

In [35]:
df = pd.read_excel(os.path.join(workdir, "testdata/test_mutations_hg19.xlsx"))
df

,Index,Gene,Transcript,Location,Pos.,Type,Nuc Change,check Nuc,Unnamed: 8,Coverage,AA Change,Annotation,Hint,web Ref.,c. HGVS,p. HGVS,mut Entry,mut Effect,TValidation,MValidation,Weighting,MutDB:Classification,MutDB:Has Weblinks,ClinVitae:Classification,ClinVitae:Reported Classification,COSMIC:Pubmed_PMID,ClinVar:Clinical Significance,ClinVar:ReviewStatus,1000Genomes:AF,1000Genomes:AVGPOST,1000Genomes:EUR_AF,dbSNP:ASS,dbSNP:COMMON,dbSNP:DSS
0,4.0,FGFR4,NM_213647,E16,103 (2118) [chr5:g.176523707 (hg19)],C,T -> C (het),OK,60.0,60% (4570) [60% (2481) / 59% (2089)],H -> H (706),"synonymous, coding sequence, exon",primer position,rs151207425 (1000Genomes; ExAC; dbSNP; gnomAD),c.2118T>C,p.His706,0/7 of 2382,NaN,NaN,NaN,distinct,likely benign,no,NaN,NaN,NaN,NaN,NaN,0.000200,NaN,0.0010,NaN,NaN,NaN
1,5.0,FGFR4,NM_213647,E16,103 (2118) [chr5:g.176523707 (hg19)],C,T -> C (het),OK,58.0,58% (3786) [58% (2074) / 58% (1712)],H -> H (706),"synonymous, coding sequence, exon",primer position,rs151207425 (1000Genomes; ExAC; dbSNP; gnomAD),c.2118T>C,p.His706,0/7 of 2382,NaN,NaN,NaN,distinct,likely benign,no,NaN,NaN,NaN,NaN,NaN,0.000200,NaN,0.0010,NaN,NaN,NaN
2,5.0,FGFR4,NM_213647,E9,105 (1162) [chr5:g.176520243 (hg19)],C,G -> A (homo),OK,99.0,99% (4939) [99% (1759) / 100% (3180)],G -> R (388),"missense, protein altering, coding sequence, exon",NaN,"COSM1567768 (COSMIC), rs351855 (1000Genomes; ClinVar; ExAC; dbSNP; gnomAD)",c.1162G>A,p.Gly388Arg,225/1122 of 2861,Polymorphismus,NaN,NaN,distinct,likely benign,no,Pathogenic,Pathogenic,NaN,Pathogenic,NaN,0.299521,NaN,0.2942,NaN,yes,NaN
3,6.0,FGFR4,NM_213647,E9,105 (1162) [chr5:g.176520243 (hg19)],C,G -> A (homo),OK,99.0,99% (4177) [99% (1491) / 100% (2686)],G -> R (388),"missense, protein altering, coding sequence, exon",NaN,"COSM1567768 (COSMIC), rs351855 (1000Genomes; ClinVar; ExAC; dbSNP; gnomAD)",c.1162G>A,p.Gly388Arg,225/1122 of 2861,Polymorphismus,NaN,NaN,distinct,likely benign,no,Pathogenic,Pathogenic,NaN,Pathogenic,NaN,0.299521,NaN,0.2942,NaN,yes,NaN
4,2.0,FGFR4,NM_213647,E12,47 (1566) [chr5:g.176522377 (hg19)],C,G -> T (het),OK,60.0,60% (4196) [61% (1860) / 59% (2336)],E -> D (522),"missense, protein altering, coding sequence, exon",NaN,NaN,c.1566G>T,p.Glu522Asp,NaN,NaN,NaN,NaN,distinct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,FGFR4,NM_213647,E12,47 (1566) [chr5:g.176522377 (hg19)],C,G -> T (het),OK,38.0,38% (2061) [38% (817) / 38% (1244)],E -> D (522),"missense, protein altering, coding sequence, exon",NaN,NaN,c.1566G>T,p.Glu522Asp,NaN,NaN,NaN,NaN,distinct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,HRAS,NM_005343,E2,44 [chr11:g.534332 (hg19)],C,C -> T (het),C=G,4.8,4.8% (349) [6% (159) / 4.1% (190)],5' UTR,"5' UTR, exon",NaN,rs41294870 (1000Genomes; ClinVar; ExAC; dbSNP; gnomAD),c.-10C>T,NaN,1/43 of 697 [c.-10C>T (NM_001130442)],NaN,NaN,NaN,distinct,benign,no,"Benign,Not Provided","Benign,not provided",NaN,Benign,NaN,0.035743,NaN,0.0457,NaN,yes,NaN
7,4.0,HRAS,NM_005343,E2,44 [chr11:g.534332 (hg19)],C,G -> A (het),C=G,29.0,29% (2459) [33% (1122) / 27% (1337)],5' UTR,"5' UTR, exon",NaN,rs41294870 (1000Genomes; ClinVar; ExAC; dbSNP; gnomAD),c.-10C>T,NaN,1/43 of 697 [c.-10C>T (NM_001130442)],NaN,NaN,NaN,distinct,benign,no,"Benign,Not Provided","Benign,not provided",NaN,Benign,NaN,0.035743,NaN,0.0457,NaN,yes,NaN
8,3.0,KEAP1,NM_203500,E3,374 (1013) [chr19:g.10602565 (hg19)],C,C -> A (het),C=G,95.0,95% (9632) [95% (4890) / 96% (4742)],S -> [STOP] (338),"stop gained, missense, protein altering, coding sequence, exon",primer position,NaN,c.1013C>A,p.Ser338Ter,NaN,NaN,NaN,NaN,distinct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,KEAP1,NM_203500,E3,374 (1013) [chr19:g.10602565 (hg19)],C,G -> T (het),C=G,50.0,50% (5186) [49% (2692) / 51% (2494)],S -> [STOP] (338),"stop gained, missense, protein altering, coding sequence, exon",primer position,NaN,c.1013C>A,p.Ser338Ter,NaN,NaN,NaN,NaN,distinct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are a lot of columns and information right now. The last rows are leftovers from the excel transformation so we get rid of them with iloc. To make things easy, we also remove every additional column right now and keep only the columns needed for annotation.
The coords are somehow stored in the `Pos.` column and the mutation in the `Nuc Change` column. So do all this with `iloc`..

In [36]:
df = df.iloc[:16, [4,6]]
# df = df.iloc[:15, :]
df

,Pos.,Nuc Change
0,103 (2118) [chr5:g.176523707 (hg19)],T -> C (het)
1,103 (2118) [chr5:g.176523707 (hg19)],T -> C (het)
2,105 (1162) [chr5:g.176520243 (hg19)],G -> A (homo)
3,105 (1162) [chr5:g.176520243 (hg19)],G -> A (homo)
4,47 (1566) [chr5:g.176522377 (hg19)],G -> T (het)
5,47 (1566) [chr5:g.176522377 (hg19)],G -> T (het)
6,44 [chr11:g.534332 (hg19)],C -> T (het)
7,44 [chr11:g.534332 (hg19)],G -> A (het)
8,374 (1013) [chr19:g.10602565 (hg19)],C -> A (het)
9,374 (1013) [chr19:g.10602565 (hg19)],G -> T (het)


### wrangle
Now, we extract the relevant information with str.extract and some smart regex. This will be different everytime and if things get too complicated consider chaining several extracts and even to modify the source file itself. Try to be efficient here!

In [37]:
df.loc[:,["Start", "Chr"]] = df['Pos.'].str.extract(r"(?P<Chr>chr[0-9]+):g\.(?P<Start>[0-9]+)")
df.loc[:,["Ref", "Alt"]] = df['Nuc Change'].str.extract(r"(?P<Ref>[ACTG]) -> (?P<Alt>[ACTG])")
df.loc[:, "End"] = df['Start']
# df = df.loc[:,["Chr", "Start", "End", "Ref", "Alt"]]
df

,Pos.,Nuc Change,Start,Chr,Ref,Alt,End
0,103 (2118) [chr5:g.176523707 (hg19)],T -> C (het),176523707,chr5,T,C,176523707
1,103 (2118) [chr5:g.176523707 (hg19)],T -> C (het),176523707,chr5,T,C,176523707
2,105 (1162) [chr5:g.176520243 (hg19)],G -> A (homo),176520243,chr5,G,A,176520243
3,105 (1162) [chr5:g.176520243 (hg19)],G -> A (homo),176520243,chr5,G,A,176520243
4,47 (1566) [chr5:g.176522377 (hg19)],G -> T (het),176522377,chr5,G,T,176522377
5,47 (1566) [chr5:g.176522377 (hg19)],G -> T (het),176522377,chr5,G,T,176522377
6,44 [chr11:g.534332 (hg19)],C -> T (het),534332,chr11,C,T,534332
7,44 [chr11:g.534332 (hg19)],G -> A (het),534332,chr11,G,A,534332
8,374 (1013) [chr19:g.10602565 (hg19)],C -> A (het),10602565,chr19,C,A,10602565
9,374 (1013) [chr19:g.10602565 (hg19)],G -> T (het),10602565,chr19,G,T,10602565


### convert to hg38
Although there are some tools that do that automatically, I still use the USCS website for conversion. For that, the coords have to be converted to the bed-format `Chr:Start-End`. This is always tedious and I use the helpers provided by the repo.
Here are the steps:
+ create the coords using `pos2bed` helper. Set option `as_string=True` and print the output for direct copying to the browser as it removes the index
+ copy to the website and create the conversion
+ reintroduce the hg38 coords into the file

In [38]:
from pyseq_utils import pos2bed
print(pos2bed(df, as_string=True))

chr5:176523707-176523707
chr5:176523707-176523707
chr5:176520243-176520243
chr5:176520243-176520243
chr5:176522377-176522377
chr5:176522377-176522377
     chr11:534332-534332
     chr11:534332-534332
 chr19:10602565-10602565
 chr19:10602565-10602565
chr1:156851382-156851382
chr1:156851382-156851382
 chr10:43613783-43613783
 chr10:43613783-43613783
   chr17:7578536-7578536
   chr17:7578536-7578536


+ Now copy the coords to the clipboard and paste into the [uscs liftover website](https://genome.ucsc.edu/cgi-bin/hgLiftOver)
+ click on `View conversions` and retrieve the file from your download folder. Should be something like `hglft_genome....be`

In [39]:
hg38 = pd.read_csv(os.path.join(home, "Downloads/hglft_genome_1181f_fb3d20.bed"), sep="\t", names=['hg38'])
hg38

,hg38
0,chr5:177096706-177096706
1,chr5:177096706-177096706
2,chr5:177093242-177093242
3,chr5:177093242-177093242
4,chr5:177095376-177095376
5,chr5:177095376-177095376
6,chr11:534332-534332
7,chr11:534332-534332
8,chr19:10491889-10491889
9,chr19:10491889-10491889


`bed2pos`reconverts the column into Chr Start End that can be reinserted into the dataframe. This only works by index, so make sure that your original df indices have not changed and are in a reset state!!

In [40]:
from pyseq_utils import bed2pos
df.loc[:, ["Chr", "Start", "End"]] = bed2pos(hg38['hg38'])
df = df.loc[:, ['Chr', 'Start', 'End', 'Ref', 'Alt']]
df

,Chr,Start,End,Ref,Alt
0,chr5,177096706,177096706,T,C
1,chr5,177096706,177096706,T,C
2,chr5,177093242,177093242,G,A
3,chr5,177093242,177093242,G,A
4,chr5,177095376,177095376,G,T
5,chr5,177095376,177095376,G,T
6,chr11,534332,534332,C,T
7,chr11,534332,534332,G,A
8,chr19,10491889,10491889,C,A
9,chr19,10491889,10491889,G,T


### annotate data with annovar
Annovar is a very comprehensive tool for annotation of mutations using various databases. See [here](https://annovar.openbioinformatics.org/en/latest/) for all you need to know. I provide a static file for genomic annotation that contains (among others) updated databases for use with annovar.
For downloading the static files, do this from the root folder:
+ `$ . setup/download_static.sh <path-to-static-folder>  # provide a folder for downloading and expanding ~40GB of data`
+ for annovar to work, the path to the static folder has to be set in an environment variable STATIC: 
    * `$ export STATIC=<path-to-static-folder>`. 
    * For making this permanent, copy this line to your .bash_profile file in your HOME folder:
    * (`$ echo "export STATIC=<path-to-static-folder>" >> "${HOME}/.bash_profile`)

This will take some time as it is downloading and unpacking a 40GB file!!

Next, you have to configure the annovar_config.yaml file, giving it
   + the absolute path to the annovar folder sitting in this repo at ./code/anno2019. This depends on where your repo is residing.
   + the path in the STATIC folder to the annovar database (only change this if you moved the humandb folder somewhere else relative to the static folder)
   + a list of databases from the annovar database to use for populating the mutations
        * see the list of currently stored databases like so:`$ ls ${STATIC}/hg38/annotation/annovar/humandb/*.txt`
        * if you want the database hg38_icgc29.txt to be used, just list "icgc29" in the yaml file

Now, can run the annovar tool (run_annovar is a convenience wrapper around the command line tool written in perl), so you only have to worry once about providing the absolute path to the annovar code folder. You can see the large 
This will take some time depending on the list and size of both the mutations and the number and size of the databases used. You can see the long command line call in the cell output.

In [41]:
from anno import run_annovar
config_file = "../configs/annovar_config.yaml"
                         
df_anno = run_annovar(df, config_file, threads=10, cleanup=False)


$ perl /Users/martinszyska/Sites/Bio/pyseq/code/anno2019/table_annovar.pl -buildver hg38 --maxgenethread 10 --thread 10 --protocol ensGene34,cytoBand,gnomad30,dbSNP154,cosmic95,icgc29,clinvar2021 --operation g,r,f,f,f,f,f -nastring "." --remove --outfile /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/annovar_out /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/anno_file.csv /Users/martinszyska/Dropbox/Icke/Work/static/hg38/annotation/annovar/humandb


NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=ensGene34

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver hg38 -dbtype ensGene34 -outfile /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/annovar_out.ensGene34 -exonsort -nofirstcodondel /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/anno_file.csv /Users/martinszyska/Dropbox/Icke/Work/static/hg38/annotation/annovar/humandb -thread 10 -maxgenethread 10>
NOTICE: Output files are written to /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/annovar_out.ensGene34.variant_function, /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/annovar_out.ensGene34.exonic_variant_function
NOTICE: the queryfile /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/anno_file.csv contains 32 lines
NOTICE: threading is disabled for gene-based annotation on file with less than 1000000

NOTICE: Processing next batch with 2 unique variants in 4 input lines
NOTICE: Database index loaded. Total number of bins is 314742 and the number of bins to be scanned is 2
NOTICE: Scanning filter database /Users/martinszyska/Dropbox/Icke/Work/static/hg38/annotation/annovar/humandb/hg38_dbSNP154.txt...Done
NOTICE: Processing next batch with 3 unique variants in 4 input lines
NOTICE: Database index loaded. Total number of bins is 314742 and the number of bins to be scanned is 2
NOTICE: Scanning filter database /Users/martinszyska/Dropbox/Icke/Work/static/hg38/annotation/annovar/humandb/hg38_dbSNP154.txt...Done
NOTICE: Processing next batch with 3 unique variants in 4 input lines
NOTICE: Database index loaded. Total number of bins is 314742 and the number of bins to be scanned is 2
NOTICE: Scanning filter database /Users/martinszyska/Dropbox/Icke/Work/static/hg38/annotation/annovar/humandb/hg38_dbSNP154.txt...Done
NOTICE: Processing next batch with 3 unique variants in 4 input lines
NOT

Chr      object
Start     int64
End       int64
Ref      object
Alt      object
dtype: object Chr                     object
Start                    int64
End                      int64
Ref                     object
Alt                     object
Func.ensGene34          object
Gene.ensGene34          object
GeneDetail.ensGene34    object
ExonicFunc.ensGene34    object
AAChange.ensGene34      object
cytoBand                object
gnomAD_exome_ALL        object
dbSNP154_ID             object
dbSNP154_AltFreq        object
dbSNP154_DB             object
Mut_ID                  object
Mut_ID_old              object
count                   object
type                    object
MutStatus               object
icgc29_ID               object
icgc29_Affected         object
CLNALLELEID             object
CLNDN                   object
CLNSIG                  object
dtype: object


NOTICE: Processing next batch with 3 unique variants in 4 input lines
NOTICE: Database index loaded. Total number of bins is 76449 and the number of bins to be scanned is 2
NOTICE: Scanning filter database /Users/martinszyska/Dropbox/Icke/Work/static/hg38/annotation/annovar/humandb/hg38_clinvar2021.txt...Done
-----------------------------------------------------------------
NOTICE: Multianno output file is written to /Users/martinszyska/Sites/Bio/pyseq/nb/anno_temp/annovar_out.hg38_multianno.txt


### The annovar result
You can inspect the output and see that, depending on the used databases you have several new columns in your output dataframe
But first, it would be wise to remove some of the columns

In [44]:
df_anno[:3]

,Chr,Start,End,Ref,Alt,Func.ensGene34,Gene.ensGene34,GeneDetail.ensGene34,ExonicFunc.ensGene34,AAChange.ensGene34,cytoBand,gnomAD_exome_ALL,dbSNP154_ID,dbSNP154_AltFreq,dbSNP154_DB,Mut_ID,Mut_ID_old,count,type,MutStatus,icgc29_ID,icgc29_Affected,CLNALLELEID,CLNDN,CLNSIG
0,chr5,177096706,177096706,T,C,exonic,FGFR4,.,synonymous SNV,"FGFR4:ENST00000393637.5:exon14:c.T1998C:p.H666H,FGFR4:ENST00000292408.9:exon16:c.T2118C:p.H706H,FGFR4:ENST00000393648.6:exon16:c.T1914C:p.H638H,FGFR4:ENST00000502906.5:exon16:c.T2118C:p.H706H",5q35.2,0.0003,rs151207425,0.0001997,1000Genomes,.,.,.,.,.,.,.,.,.,.
1,chr5,177096706,177096706,T,C,exonic,FGFR4,.,synonymous SNV,"FGFR4:ENST00000393637.5:exon14:c.T1998C:p.H666H,FGFR4:ENST00000292408.9:exon16:c.T2118C:p.H706H,FGFR4:ENST00000393648.6:exon16:c.T1914C:p.H638H,FGFR4:ENST00000502906.5:exon16:c.T2118C:p.H706H",5q35.2,0.0003,rs151207425,0.0001997,1000Genomes,.,.,.,.,.,.,.,.,.,.
2,chr5,177093242,177093242,G,A,exonic,FGFR4,.,nonsynonymous SNV,"FGFR4:ENST00000292408.9:exon9:c.G1162A:p.G388R,FGFR4:ENST00000502906.5:exon9:c.G1162A:p.G388R",5q35.2,0.2632,rs351855,0.2995,1000Genomes,COSV52800825,COSM1567768,128,4x(adenocarcinoma@colon)+24x(alveolar@striated_muscle)+4x(carcinoma@breast)+8x(carcinoma@lung)+8x(embryonal@striated_muscle)+4x(fibroblastic@meninges)+32x(haemangioblastoma@blood_vessel)+8x(mening...,ReportedElsewhere,MU153762,2/170,31365,See_cases|Cancer_progression_and_tumor_cell_motility,Uncertain_significance


In [46]:
df_anno.columns
df_anno = df_anno.loc[:, ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.ensGene34', 'Gene.ensGene34',
        'ExonicFunc.ensGene34',
       'cytoBand', 'gnomAD_exome_ALL', 'dbSNP154_AltFreq',
       'Mut_ID', 'count', 'type',
       'icgc29_ID', 'icgc29_Affected', 'CLNSIG']]
df_anno

Index(['Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.ensGene34', 'Gene.ensGene34',
       'ExonicFunc.ensGene34', 'cytoBand', 'gnomAD_exome_ALL',
       'dbSNP154_AltFreq', 'Mut_ID', 'count', 'type', 'icgc29_ID',
       'icgc29_Affected', 'CLNSIG'],
      dtype='object')

,Chr,Start,End,Ref,Alt,Func.ensGene34,Gene.ensGene34,ExonicFunc.ensGene34,cytoBand,gnomAD_exome_ALL,dbSNP154_AltFreq,Mut_ID,count,type,icgc29_ID,icgc29_Affected,CLNSIG
0,chr5,177096706,177096706,T,C,exonic,FGFR4,synonymous SNV,5q35.2,0.0003,0.0001997,.,.,.,.,.,.
1,chr5,177096706,177096706,T,C,exonic,FGFR4,synonymous SNV,5q35.2,0.0003,0.0001997,.,.,.,.,.,.
2,chr5,177093242,177093242,G,A,exonic,FGFR4,nonsynonymous SNV,5q35.2,0.2632,0.2995,COSV52800825,128,4x(adenocarcinoma@colon)+24x(alveolar@striated_muscle)+4x(carcinoma@breast)+8x(carcinoma@lung)+8x(embryonal@striated_muscle)+4x(fibroblastic@meninges)+32x(haemangioblastoma@blood_vessel)+8x(mening...,MU153762,2/170,Uncertain_significance
3,chr5,177093242,177093242,G,A,exonic,FGFR4,nonsynonymous SNV,5q35.2,0.2632,0.2995,COSV52800825,128,4x(adenocarcinoma@colon)+24x(alveolar@striated_muscle)+4x(carcinoma@breast)+8x(carcinoma@lung)+8x(embryonal@striated_muscle)+4x(fibroblastic@meninges)+32x(haemangioblastoma@blood_vessel)+8x(mening...,MU153762,2/170,Uncertain_significance
4,chr5,177095376,177095376,G,T,exonic,FGFR4,nonsynonymous SNV,5q35.2,.,.,COSV99450816,4,4x(adenocarcinoma@lung),MU131464768,1/516,.
5,chr5,177095376,177095376,G,T,exonic,FGFR4,nonsynonymous SNV,5q35.2,.,.,COSV99450816,4,4x(adenocarcinoma@lung),MU131464768,1/516,.
6,chr11,534332,534332,C,T,UTR5,HRAS,.,11p15.5,.,.,.,.,.,.,.,.
7,chr11,534332,534332,G,A,UTR5,HRAS,.,11p15.5,0.0365,0.03574,COSV54240784,10,10x(haemangioblastoma@blood_vessel),.,.,Benign
8,chr19,10491889,10491889,C,A,exonic,KEAP1,nonsynonymous SNV,19p13.2,.,.,COSV50276845,6,2x(ER-PR-positive_carcinoma@breast)+2x(adenocarcinoma@gallbladder)+2x(squamous_cell_carcinoma@lung),MU129512036,1/531,.
9,chr19,10491889,10491889,G,T,exonic,KEAP1,stopgain,19p13.2,.,.,.,.,.,.,.,.


### compute clinscore